## Value of an European Call Option

The [Black-Scholes-Merton](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model) model is a 
mathematical model applicable in financial markets containing derivative investment instruments for 
the theoretical estimation of European-style options' price. According to such model, 
__the index level at maturity__ of a call option can be modeled as it follows with z normally distributed

$$S_T(z)=S_0e^{(r-0.5\sigma^2)T+\sigma\sqrt{T}z}$$

where 
* $S_0$ is the initial stock index level
* K is the strike price of the European call option
* T is the Time-to-maturity in year 
* r is the riskless short rate 
* $\sigma^2$ is the volatility

Let's assume the following values 
* $S_0 = 105$
* K = 109 
* T = 1 
* r = 6% 
* $\sigma^2$ = 22%

In order to calculate the present value of the call option, it is possible to apply a Monte Carlo approach
according to the following procedure: 
* Let's consider R realizations of the random variable z from a standard normal distribution 
* Compute related index levels at maturity $S_T(i)$ for $z(i)\in\{1,2,...,R\}$ 
* Compute the call option value $h_T(i)=\max{(S_T(i)–K,0)}$. 
* Compute the __present value__ of the call option, i.e. 

$$C_0=e^{-rT}{R}^{-1}\sum_{i=1}^{R} h_T{(i)}$$



In [1]:
import numpy as np

def bsm(S0,r,sigma,T,K,R = 100000 , seed=500):
    np.random.seed(seed)
    z = np.random.standard_normal(R) 
    ST = S0 * np.exp(( r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * z)
    hT = np.maximum(ST - K, 0)
    C0 = np.exp(-r * T) * np.sum(hT) / R
    return C0

In [2]:
import time
tm = time.time()
C0 = bsm(S0=105,r=0.06,sigma=0.22,T=1.0,K=109,R = 100000 , seed=500)
pm = time.time() - tm
print("Value of the European Call Option is {0:.4g}".format(C0)+" - time[{0:.4g} secs]".format(pm))

Value of the European Call Option is 10.31 - time[0.008368 secs]


Let's see how much time is necessary for 70,000,000 iterations intead of 100,000 iterations. 

In [3]:
tm = time.time()
C0 = bsm(S0=105,r=0.06,sigma=0.22,T=1.0,K=109,R = 70000000 , seed=500)
pm = time.time() - tm
print("Value of the European Call Option is {0:.4g}".format(C0)+" - time[{0:.4g} secs]".format(pm))

Value of the European Call Option is 10.32 - time[5.455 secs]


Let's see how we can speed up the computation with the __numexpr__ package. 

In [4]:
import numexpr as ne
def bsm_ne(S0,r,sigma,T,K,R = 70000000 , seed=500):
    np.random.seed(seed)
    z = np.random.standard_normal(R) 
    ST = ne.evaluate('S0 * exp(( r - 0.5 * sigma ** 2) * T + sigma * sqrt(T) * z)')
    hT = np.maximum(ST - K, 0)
    C0 = np.exp(-r * T) * np.sum(hT) / R
    return C0

In [5]:
tm = time.time()
C0 = bsm_ne(S0=105,r=0.06,sigma=0.22,T=1.0,K=109,R = 70000000 , seed=500)
pm = time.time() - tm
print("Value of the European Call Option is {0:.4g}".format(C0)+" - time[{0:.4g} secs]".format(pm))

Value of the European Call Option is 10.32 - time[4.196 secs]
